<a href="https://colab.research.google.com/github/sthalles/SimCLR/blob/simclr-refactor/feature_eval/mini_batch_logistic_regression_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
import torch
import sys
import numpy as np
import os
import yaml
import matplotlib.pyplot as plt
import torchvision
from torchattacks import PGD, FGSM

In [2]:
%pip install gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 97 kB 4.7 MB/s eta 0:00:011
  Created wheel for gdown: filename=gdown-4.2.0-py3-none-any.whl size=14262 sha256=8101fff7be5acacedaf8656f669438b1a01b1abebf2e050b9deb76cce3ac96b8
  Stored in directory: /home/tj2467/.cache/pip/wheels/8c/17/ff/58721d1fabdb87c21a0529948cf39e2be9af90ddbe4ad65944
Successfully built gdown
Note: you may need to restart the kernel to use updated packages.


In [3]:
def get_file_id_by_model(folder_name):
  file_id = {'resnet18_100-epochs_stl10': '14_nH2FkyKbt61cieQDiSbBVNP8-gtwgF',
             'resnet18_100-epochs_cifar10': '1lc2aoVtrAetGn0PnTkOyFzPCIucOJq7C',
             'resnet50_50-epochs_stl10': '1ByTKAUsdm_X7tLcii6oAEl5qFRqRMZSu'}
  return file_id.get(folder_name, "Model not found.")

In [4]:
folder_name = 'resnet50_50-epochs_stl10'
file_id = get_file_id_by_model(folder_name)
print(folder_name, file_id)

resnet50_50-epochs_stl10 1ByTKAUsdm_X7tLcii6oAEl5qFRqRMZSu


In [14]:
# download and extract model files
os.system('gdown https://drive.google.com/uc?id={}'.format(file_id))
os.system('unzip {}'.format(folder_name))
!ls

checkpoint_0040.pth.tar
config.yml
events.out.tfevents.1610927742.4cb2c837708d.2694093.0
resnet50_50-epochs_stl10.zip
sample_data
training.log


In [33]:
from torch.utils.data import DataLoader, TensorDataset, SubsetRandomSampler
import torchvision.transforms as transforms
from torchvision import datasets

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

Using device: cuda


In [35]:
def get_stl10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.CIFAR10('./data', split='train', download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=batch_size // 16, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.CIFAR10('./data', split='test', download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=batch_size // 16, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

def get_cifar10_data_loaders(download, shuffle=False, batch_size=256, frac=1):
  train_dataset = datasets.CIFAR10('./data', train=True, download=download,
                                  transform=transforms.ToTensor())

  train_indices = np.random.choice(len(train_dataset), int(frac*len(train_dataset)))

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=batch_size // 16, drop_last=False, shuffle=shuffle, sampler=SubsetRandomSampler(train_indices))
  
  test_dataset = datasets.CIFAR10('./data', train=False, download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=batch_size // 16, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

In [5]:
with open(os.path.join('./config.yml')) as file:
  config = yaml.load(file)

FileNotFoundError: [Errno 2] No such file or directory: './config.yml'

In [214]:
model = torchvision.models.resnet18(pretrained=False, num_classes=10).to(device)
# if config.arch == 'resnet18':
#   model = torchvision.models.resnet18(pretrained=False, num_classes=10).to(device)
# elif config.arch == 'resnet50':
#   model = torchvision.models.resnet50(pretrained=False, num_classes=10).to(device)

In [197]:
checkpoint = torch.load('../runs/Dec18_23-00-28_instance-1/checkpoint_0200.pth.tar', map_location=device)
state_dict = checkpoint['state_dict']

for k in list(state_dict.keys()):

  if k.startswith('backbone.'):
    if k.startswith('backbone') and not k.startswith('backbone.fc'):
      # remove prefix
      state_dict[k[len("backbone."):]] = state_dict[k]
  del state_dict[k]

In [198]:
log = model.load_state_dict(state_dict, strict=False)
assert log.missing_keys == ['fc.weight', 'fc.bias']

In [199]:
train_loader, test_loader = get_cifar10_data_loaders(download=True)
train_loader_1, test_loader = get_cifar10_data_loaders(download=True, frac=0.01)
train_loader_10, test_loader = get_cifar10_data_loaders(download=True, frac=0.1)
train_loader_30, test_loader = get_cifar10_data_loaders(download=True, frac=0.3)
# if config.dataset_name == 'cifar10':
#   train_loader, test_loader = get_cifar10_data_loaders(download=True)
# elif config.dataset_name == 'stl10':
#   train_loader, test_loader = get_stl10_data_loaders(download=True)
# print("Dataset:", config.dataset_name)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [200]:
# freeze all layers but the last fc
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        param.requires_grad = False

In [215]:
parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
# assert len(parameters) == 2  # fc.weight, fc.bias

In [216]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.0008)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [217]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [204]:
epochs = 20
for epoch in range(epochs):
  top1_train_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(train_loader_1):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
    loss = criterion(logits, y_batch)
    top1 = accuracy(logits, y_batch, topk=(1,))
    top1_train_accuracy += top1[0]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  top1_train_accuracy /= (counter + 1)
  top1_accuracy = 0
  top5_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(test_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
  
    top1, top5 = accuracy(logits, y_batch, topk=(1,5))
    top1_accuracy += top1[0]
    top5_accuracy += top5[0]
  
  top1_accuracy /= (counter + 1)
  top5_accuracy /= (counter + 1)
  print(f"Epoch {epoch}\tTop1 Train accuracy {top1_train_accuracy.item()}\tTop1 Test accuracy: {top1_accuracy.item()}\tTop5 test acc: {top5_accuracy.item()}")

Epoch 0	Top1 Train accuracy 11.939677238464355	Top1 Test accuracy: 10.025275230407715	Top5 test acc: 49.78515625


Exception ignored in: <function _releaseLock at 0x7f6b996e4290>
Traceback (most recent call last):
  File "/home/tj2467/miniconda3/envs/simclr/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt


RuntimeError: DataLoader worker (pid(s) 12978, 12979, 12980, 12981) exited unexpectedly

In [218]:
epochs = 50
tl = train_loader
for epoch in range(epochs):
  top1_train_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(tl):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
    loss = criterion(logits, y_batch)
    top1 = accuracy(logits, y_batch, topk=(1,))
    top1_train_accuracy += top1[0]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # atk = PGD(model, eps=8/255, alpha=2/255, steps=7)
  atk = FGSM(model)
  atk.set_return_type('int') # Save as integer.
  atk.save(data_loader=tl, save_path="./data/cifar10_fgsm.pt", verbose=True)
  adv_images, adv_labels = torch.load("./data/cifar10_fgsm.pt")
  adv_data = TensorDataset(adv_images.float()/255, adv_labels)
  adv_loader = DataLoader(adv_data, batch_size=128, shuffle=False)
  
  for counter, (x_batch, y_batch) in enumerate(adv_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
    loss = criterion(logits, y_batch)
    top1 = accuracy(logits, y_batch, topk=(1,))
    top1_train_accuracy += top1[0]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  top1_train_accuracy /= (counter + 1)
  top1_accuracy = 0
  top5_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(test_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
  
    top1, top5 = accuracy(logits, y_batch, topk=(1,5))
    top1_accuracy += top1[0]
    top5_accuracy += top5[0]
  
  top1_accuracy /= (counter + 1)
  top5_accuracy /= (counter + 1)
  print(f"Epoch {epoch}\tTop1 Train accuracy {top1_train_accuracy.item()}\tTop1 Test accuracy: {top1_accuracy.item()}\tTop5 test acc: {top5_accuracy.item()}")

- Save progress: 100.00 % / Robust accuracy: 12.67 % / L2: 0.38640 (0.051 it/s) 	
- Save complete!
Epoch 0	Top1 Train accuracy 80.861572265625	Top1 Test accuracy: 58.46967315673828	Top5 test acc: 95.36420440673828
- Save progress: 100.00 % / Robust accuracy: 35.33 % / L2: 0.38656 (0.052 it/s) 	
- Save complete!
Epoch 1	Top1 Train accuracy 107.8472671508789	Top1 Test accuracy: 62.86707305908203	Top5 test acc: 96.40050506591797
- Save progress: 100.00 % / Robust accuracy: 60.56 % / L2: 0.38679 (0.048 it/s) 	
- Save complete!
Epoch 2	Top1 Train accuracy 122.39189910888672	Top1 Test accuracy: 63.60868453979492	Top5 test acc: 96.56824493408203
- Save progress: 100.00 % / Robust accuracy: 73.78 % / L2: 0.38704 (0.051 it/s) 	
- Save complete!
Epoch 3	Top1 Train accuracy 130.46035766601562	Top1 Test accuracy: 65.04595184326172	Top5 test acc: 96.45048522949219
- Save progress: 100.00 % / Robust accuracy: 86.00 % / L2: 0.38728 (0.049 it/s) 	
- Save complete!
Epoch 4	Top1 Train accuracy 135.12767

KeyboardInterrupt: 

In [ ]:
from torchattacks import PGD, FGSM

atk = PGD(model, eps=8/255, alpha=2/255, steps=7)
atk.set_return_type('int') # Save as integer.
atk.save(data_loader=test_loader, save_path="./data/cifar10_pgd.pt", verbose=True)

In [ ]:
adv_images, adv_labels = torch.load("./data/cifar10_pgd.pt")
adv_data = TensorDataset(adv_images.float()/255, adv_labels)
adv_loader = DataLoader(adv_data, batch_size=128, shuffle=False)

In [219]:
from torchattacks import PGD, FGSM

# atk = PGD(model, eps=8/255, alpha=2/255, steps=7)
atk = FGSM(model)
atk.set_return_type('int') # Save as integer.
atk.save(data_loader=test_loader, save_path="./data/cifar10_fgsm.pt", verbose=True)

- Save progress: 100.00 % / Robust accuracy: 43.11 % / L2: 0.38660 (0.130 it/s) 	
- Save complete!


In [220]:
adv_images, adv_labels = torch.load("./data/cifar10_fgsm.pt")
adv_data = TensorDataset(adv_images.float()/255, adv_labels)
adv_loader = DataLoader(adv_data, batch_size=128, shuffle=False)

In [221]:
model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Standard accuracy: %.2f %%' % (100 * float(correct) / total))

Standard accuracy: 68.48 %


In [222]:
model.eval()

correct = 0
total = 0

for images, labels in adv_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 46.61 %
